**Set environment**

In [1]:
suppressMessages(suppressWarnings(source("../config/config_sing.R")))
show_env()

You are in Singularity: singularity_proj_encode_fcc 
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei 
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out 
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code 
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC 
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc 
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log 


**Check data files**

In [2]:
FOLDER_A = "annotation_fcc_table"
FD_REG_A = file.path(
    FD_RES, 
    "results", 
    "region", 
    FOLDER_A)

FOLDER_B = "annotation_chromHMM"
FD_REG_B = file.path(
    FD_RES, 
    "results", 
    "region", 
    FOLDER_B)

FD_REG_C = file.path(
    FD_REG_A,
    FOLDER_B
)

FD_OUT = file.path(
    FD_REG_A,
    "summary"
)

In [3]:
fdiry  = FD_REG_A
fnames = dir(fdiry)
for(fname in fnames){print(fname)}

[1] "annotation_ccres"
[1] "annotation_ccres_silencer"
[1] "annotation_chromHMM"
[1] "annotation_tss_pol2"
[1] "description.tsv"
[1] "fcc_table.starrmpra.crispri.atac.concat.bed.gz"
[1] "fcc_table.starrmpra.crispri.atac.e2g_benchmark.concat.bed.gz"
[1] "fcc_table.starrmpra.crispri.atac.e2g_benchmark.e2g_prediction.concat.bed.gz"
[1] "fcc_table.starrmpra.crispri.atac.e2g_benchmark.e2g_prediction.merge.bed.gz"
[1] "fcc_table.starrmpra.crispri.atac.e2g_benchmark.e2g_prediction.merge.tsv"
[1] "fcc_table.starrmpra.crispri.atac.e2g_benchmark.merge.bed.gz"
[1] "fcc_table.starrmpra.crispri.atac.e2g_benchmark.merge.tsv"
[1] "fcc_table.starrmpra.crispri.atac.merge.bed.gz"
[1] "fcc_table.starrmpra.crispri.atac.merge.tsv"
[1] "summary"
[1] "z_summary"


In [4]:
fdiry  = FD_REG_B
fnames = dir(fdiry)
for(fname in fnames){print(fname)}

[1] "ChIP_ENCSR000EWA-ENCSR000AKP-ENCSR000EWC-ENCSR000DWB-ENCSR000EWB-ENCSR000APE.bed"
[1] "description.tsv"
[1] "K562.ENCSR365YNI.ENCFF106BGJ.ChromHMM.bed.gz"


In [5]:
fdiry  = FD_REG_C
fnames = dir(fdiry)
for(fname in fnames){print(fname)}

[1] "peak.annotation.K562.ENCSR365YNI.ENCFF106BGJ.ChromHMM.bed.gz"


## Import data

In [6]:
### set file paths
fdiry = FD_REG_A
fname = "description.tsv"
fpath = file.path(fdiry, fname)

### read table
dat = read_tsv(fpath, show_col_types = FALSE)
vec = dat$Name
vec[1:3] = paste(vec[1:3], "A", sep="_")

### assign and show
vec_txt_cnames_reg_A = vec
print(vec)
dat

[1] "Chrom_A"     "Start_A"     "End_A"       "Assay_Type"  "Assay_Label"


Name,Description
<chr>,<chr>
Chrom,Chromosome
Start,Start
End,End
Assay_Type,Assay type
Assay_Label,Assay label


In [7]:
### set file paths
fdiry = FD_REG_B
fname = "description.tsv"
fpath = file.path(fdiry, fname)

### read table
dat = read_tsv(fpath, show_col_types = FALSE)
vec = dat$Name
vec[1:6] = paste(vec[1:6], "B", sep="_")

### assign and show
vec_txt_cnames_reg_B = vec
print(vec)
dat

[1] "Chrom_B"    "Start_B"    "End_B"      "Name_B"     "Score_B"   
[6] "Strand_B"   "ThickStart" "ThickEnd"   "ItemRgb"   


Name,Description
<chr>,<chr>
Chrom,Chromosome
Start,Start position
End,End position
Name,ChromHMM Label; 15 state ChIP-seq model
Score,Score
Strand,[+-.]; Use '.' if no strand is assigned.
ThickStart,The starting position at which the feature is drawn thickly
ThickEnd,The ending position at which the feature is drawn thickly
ItemRgb,"An RGB value of the form R,G,B (e.g. 255,0,0)"


In [8]:
### init: set column names
vec_txt_cnames = c(
    vec_txt_cnames_reg_A,
    vec_txt_cnames_reg_B,
    "Overlap"
)

### init: set file path
fdiry = FD_REG_C
fname = "peak.annotation.K562.ENCSR365YNI.ENCFF106BGJ.ChromHMM.bed.gz"
fpath = file.path(fdiry, fname)

### init: annotation and label
annotations = "ChromHMM"
names(fnames)      = annotations
names(annotations) = annotations
print(fnames)

                                                      ChromHMM 
"peak.annotation.K562.ENCSR365YNI.ENCFF106BGJ.ChromHMM.bed.gz" 


In [12]:
### Import data
lst = lapply(annotations, function(annotation){
    ### set file path
    fname = fnames[annotation]
    fpath = file.path(fdiry, fname)
    
    ### read data
    dat = read_tsv(fpath, col_names = vec_txt_cnames, show_col_types = FALSE)
    dat = dat %>% dplyr::mutate(
        Annotation = annotation,
        Region_A   = paste0(Chrom_A, ":", Start_A, "-", End_A),
        Region_B   = paste0(Chrom_B, ":", Start_B, "-", End_B),
        Label_B    = Name_B
    )
    return(dat)
})

### assign and show
lst_peak_annot_import = lst
print(length(lst))
cat("=========================\n")
print(names(lst))
cat("=========================\n")
head(lst[[1]])

[1] 1
[1] "ChromHMM"


Chrom_A,Start_A,End_A,Assay_Type,Assay_Label,Chrom_B,Start_B,End_B,Name_B,Score_B,Strand_B,ThickStart,ThickEnd,ItemRgb,Overlap,Annotation,Region_A,Region_B,Label_B
<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>
chr1,10015,10442,ATAC,ATAC,chr1,0,16000,Quies,1,.,0,16000,"220,220,220",427,ChromHMM,chr1:10015-10442,chr1:0-16000,Quies
chr1,14253,14645,ATAC,ATAC,chr1,0,16000,Quies,1,.,0,16000,"220,220,220",392,ChromHMM,chr1:14253-14645,chr1:0-16000,Quies
chr1,16015,16477,ATAC,ATAC,chr1,16000,16200,TxWk,1,.,16000,16200,"63,154,80",185,ChromHMM,chr1:16015-16477,chr1:16000-16200,TxWk
chr1,17237,17772,ATAC,ATAC,chr1,17400,17600,TxWk,1,.,17400,17600,"63,154,80",200,ChromHMM,chr1:17237-17772,chr1:17400-17600,TxWk
chr1,28903,29613,ATAC,ATAC,chr1,17600,118400,Quies,1,.,17600,118400,"220,220,220",710,ChromHMM,chr1:28903-29613,chr1:17600-118400,Quies
chr1,30803,31072,ATAC,ATAC,chr1,17600,118400,Quies,1,.,17600,118400,"220,220,220",269,ChromHMM,chr1:30803-31072,chr1:17600-118400,Quies


In [14]:
dat = lst[[1]]
table(dat$Score_B)


     1 
372559 

## Arrange the columns

In [15]:
### init
lst = lst_peak_annot_import

lst = lapply(lst, function(dat){
    ### Select
    dat = dat %>%
        dplyr::select(
            Chrom_A, Start_A, End_A, Region_A, 
            Assay_Type, Assay_Label,
            Annotation,
            Chrom_B, Start_B, End_B, Region_B,
            Label_B)

    ### Arrange
    dat = dat %>% 
        dplyr::arrange(Chrom_A,  Start_A,  End_A)
    return(dat)
})

### assign and show
lst_peak_annot_arrange = lst
print(length(lst))
cat("=========================\n")
print(names(lst))
cat("=========================\n")
head(lst[[1]])

[1] 1
[1] "ChromHMM"


Chrom_A,Start_A,End_A,Region_A,Assay_Type,Assay_Label,Annotation,Chrom_B,Start_B,End_B,Region_B,Label_B
<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>
chr1,10015,10442,chr1:10015-10442,ATAC,ATAC,ChromHMM,chr1,0,16000,chr1:0-16000,Quies
chr1,14253,14645,chr1:14253-14645,ATAC,ATAC,ChromHMM,chr1,0,16000,chr1:0-16000,Quies
chr1,16015,16477,chr1:16015-16477,ATAC,ATAC,ChromHMM,chr1,16000,16200,chr1:16000-16200,TxWk
chr1,17237,17772,chr1:17237-17772,ATAC,ATAC,ChromHMM,chr1,17400,17600,chr1:17400-17600,TxWk
chr1,28903,29613,chr1:28903-29613,ATAC,ATAC,ChromHMM,chr1,17600,118400,chr1:17600-118400,Quies
chr1,30803,31072,chr1:30803-31072,ATAC,ATAC,ChromHMM,chr1,17600,118400,chr1:17600-118400,Quies


## Save results

In [16]:
FD_OUT

[1] "/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_fcc_table/summary"

In [17]:
lst = lst_peak_annot_arrange
for (idn in names(lst)){
    ### set file path
    fdiry = FD_OUT
    fname = paste("peak.annotation", idn, "tsv", sep=".")
    fpath = file.path(fdiry, fname)
    print(fname)
    flush.console()

    ### write table
    dat = lst[[idn]]
    write_tsv(dat, fpath)
}

[1] "peak.annotation.ChromHMM.tsv"
